In [4]:
!pip install keras


In [19]:
import numpy as np
from tensorflow import keras
 
import os
from pathlib import Path
import pandas as pd

In [20]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='', multiple=False)
display(uploader)

FileUpload(value={}, description='Upload')

In [21]:

data_path = "C:\\Users\\Lenovo\\OneDrive - Egyptian E-Learning University\\Documents\\shahd\\ara.txt"
with open(data_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [22]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path =" ara.txt"


In [23]:
data_path = r"C:\\Users\\Lenovo\\OneDrive - Egyptian E-Learning University\\Documents\\shahd\\ara.txt"
with open(data_path, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.read().split('\n')
    print(len(lines))
    for i in range(5):
        print(lines[i])


12524
Hi.	مرحبًا.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)
Run!	اركض!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1245450 (saeb)
Duck!	اخفض رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036391 (KeEichi)
Duck!	اخفضي رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036392 (KeEichi)
Duck!	اخفضوا رؤوسكم!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036393 (KeEichi)


In [24]:
with open (data_path, "r" , encoding='utf-8') as f:
    lines = f.read().split('\n')
for i in lines[:min(num_samples,len(lines)-1)]:
  input_text, target_text, _ = i.split('\t')
  target_text = '\t' + target_text + '\n'
  print(target_text)

	مرحبًا.

	اركض!

	اخفض رأسك!

	اخفضي رأسك!

	اخفضوا رؤوسكم!

	النجدة!

	اقفز!

	قف!

	توقف !

	إنتظر

	داوم.

	استمر.

	مرحباً.

	أهلاً.

	مرحباً.

	تعجّل!

	استعجل!

	انا اري

	أنا فُزت!

	استرح.

	ابتسم.

	في صحتك.

	هل فهمت؟

	ركض.

	أعرف.

	أعلم ذلك.

	أنا أعلم

	أنا في 19

	أنا بخير.

	استمع

	غير معقول!

	حقاً؟

	شكراً!

	شكرا.

	لماذا أنا؟

	رائع!

	خذ راحتك.

	أغرب عن وجهي

	هاتفني.

	اتصل بي.

	تفضل بالدخول.

	تعال إلى الداخل

	بالله عليك!

	هيا

	هيّا

	اخرج من هنا!

	أُخرج!

	اخرج!

	اتركني و شأني.

	اذهب بعيداً.

	ارحل.

	مع السلامة.

	إلى اللقاء

	إنتظر

	لقد أتى.

	هو يجري

	ساعدني!

	النجدة! ساعدني!

	ساعدوني

	انتظر.

	أنا موافق

	أنا حزين.

	أنا أيضاً.

	أنا كذلك

	اخرس!

	اصمت!

	اسكت!

	أغلق فمك!

	أوقفه

	خذه

	أخبرني

	توم فاز.

	لقد ربح توم.

	استيقظ!

	أهلاً و سهلاً!

	مرحباً بك!

	اهلا وسهلا

	مرحبا!

	من فاز؟

	من الذي ربح؟

	لم لا؟

	لما لا؟

	استمتع بوقتك.

	أسرعا.

	لقد نسيت.

	فهمتُهُ.

	فهمتُها.

	فَهمتُ ذلك.

	أستخدمه.

	سأدفع أنا.

	أنا مشغول.

	إنني مش

In [25]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), 54, num_decoder_tokens),  # استخدام القيمة الجديدة
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0
    import pickle

with open("max_lengths.pkl", "wb") as f:
    pickle.dump({
        "max_encoder_seq_length": max_encoder_seq_length,
        "max_decoder_seq_length": max_decoder_seq_length
    }, f)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 36
Max sequence length for outputs: 54


In [26]:
target_text

'\tنصحتْه ألّا يأكل كثيراً.\n'

In [27]:
input_characters

[' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '€']

In [28]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), 54, num_decoder_tokens),  # استخدام القيمة الجديدة
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
import pickle


with open("input_token_index.pkl", "wb") as f:
    pickle.dump(input_token_index, f)

with open("target_token_index.pkl", "wb") as f:
    pickle.dump(target_token_index, f)
    print("="*50)
print("input_token_index:")
for char, idx in input_token_index.items():
    print(f"{char}: {idx}")

print("\n" + "="*50)
print("target_token_index:")
for char, idx in target_token_index.items():
    print(f"{char}: {idx}")

input_token_index:
 : 0
!: 1
": 2
': 3
,: 4
-: 5
.: 6
0: 7
1: 8
2: 9
3: 10
4: 11
5: 12
6: 13
7: 14
8: 15
9: 16
:: 17
?: 18
A: 19
B: 20
C: 21
D: 22
E: 23
F: 24
G: 25
H: 26
I: 27
J: 28
K: 29
L: 30
M: 31
N: 32
O: 33
P: 34
Q: 35
R: 36
S: 37
T: 38
U: 39
V: 40
W: 41
X: 42
Y: 43
a: 44
b: 45
c: 46
d: 47
e: 48
f: 49
g: 50
h: 51
i: 52
j: 53
k: 54
l: 55
m: 56
n: 57
o: 58
p: 59
q: 60
r: 61
s: 62
t: 63
u: 64
v: 65
w: 66
x: 67
y: 68
z: 69
€: 70

target_token_index:
	: 0

: 1
 : 2
!: 3
": 4
(: 5
): 6
,: 7
-: 8
.: 9
/: 10
0: 11
1: 12
2: 13
3: 14
5: 15
6: 16
7: 17
9: 18
:: 19
?: 20
F: 21
M: 22
P: 23
\: 24
e: 25
f: 26
i: 27
o: 28
r: 29
x: 30
،: 31
؟: 32
ء: 33
آ: 34
أ: 35
ؤ: 36
إ: 37
ئ: 38
ا: 39
ب: 40
ة: 41
ت: 42
ث: 43
ج: 44
ح: 45
خ: 46
د: 47
ذ: 48
ر: 49
ز: 50
س: 51
ش: 52
ص: 53
ض: 54
ط: 55
ظ: 56
ع: 57
غ: 58
ـ: 59
ف: 60
ق: 61
ك: 62
ل: 63
م: 64
ن: 65
ه: 66
و: 67
ى: 68
ي: 69
ً: 70
ٌ: 71
ٍ: 72
َ: 73
ُ: 74
ِ: 75
ّ: 76
ْ: 77
٠: 78
١: 79
٢: 80
٣: 81
٤: 82
٥: 83
٦: 84
٨: 85
٩: 86
ٱ: 87
ی: 88
۰: 89
۱: 90
۹: 91
‍:

In [29]:
encoder_input_data.shape

(10000, 36, 71)

one-hot


In [30]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  # print(input_text)
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0
  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
  for t, char in enumerate(target_text):

    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:

        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0




In [31]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'X': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '€': 70}

model

In [32]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))


decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

train model

In [33]:
import os
from tensorflow.keras.models import load_model, save_model

model_path = "C:\\Users\\Lenovo\\OneDrive - Egyptian E-Learning University\\Documents\\shahd\\s2s_model.keras"

if not os.path.exists(model_path):
    print("Training the model for the first time...")
    model.compile(
        optimizer="rmsprop", 
        loss="categorical_crossentropy", 
        metrics=["accuracy"]
    )
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=1000,
        validation_split=0.2,
    )
    model.save(model_path)  
    print("Model saved to disk.")


else:
    print("Loading pre-trained model...")
    model = load_model(model_path)
    print("Model loaded from disk.")

Loading pre-trained model...
Model loaded from disk.


In [34]:
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
metadata = {
    "num_decoder_tokens": num_decoder_tokens,
    "target_token_index": target_token_index,
    "reverse_target_char_index": reverse_target_char_index,
    "max_decoder_seq_length": max_decoder_seq_length
}

with open("translation_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ Metadata saved to translation_metadata.pkl")


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

✅ Metadata saved to translation_metadata.pkl


In [35]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: مرحبًا.

-
Input sentence: Run!
Decoded sentence: اركض!

-
Input sentence: Duck!
Decoded sentence: اخفضوا رؤوسكم!

-
Input sentence: Duck!
Decoded sentence: اخفضوا رؤوسكم!

-
Input sentence: Duck!
Decoded sentence: اخفضوا رؤوسكم!

-
Input sentence: Help!
Decoded sentence: النجدة!

-
Input sentence: Jump!
Decoded sentence: اقفز!

-
Input sentence: Stop!
Decoded sentence: توقف !

-
Input sentence: Stop!
Decoded sentence: توقف !

-
Input sentence: Wait!
Decoded sentence: إنتظر

-
Input sentence: Go on.
Decoded sentence: استمر.

-
Input sentence: Go on.
Decoded sentence: استمر.

-
Input sentence: Hello!
Decoded sentence: مرحباً.

-
Input sentence: Hello.
Decoded sentence: مرحباً.

-
Input sentence: Hello.
Decoded sentence: مرحباً.

-
Input sentence: Hurry!
Decoded sentence: استعجل!

-
Input sentence: Hurry!
Decoded sentence: استعجل!

-
Input sentence: I see.
Decoded sentence: انا اري

-
Input sentence: I won!
Decoded sentence: أنا فُزت!

-
Input sent

In [36]:
import pandas as pd

# تحميل بيانات الترجمة
data_path = "C:\\Users\\Lenovo\\OneDrive - Egyptian E-Learning University\\Documents\\shahd\\ara.txt"
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# استخراج النصوص العربية ومعالجاتها
target_lengths = []
for line in lines[:min(num_samples, len(lines)-1)]:
    try:
        _, target_text, _ = line.split('\t')
        processed_target = '\t' + target_text + '\n'  # إضافة الرموز الخاصة
        target_lengths.append(len(processed_target))
    except:
        continue

max_decoder_seq_length = max(target_lengths)
print(f"Max decoder sequence length: {max_decoder_seq_length}")

Max decoder sequence length: 54
